# 实验参数设置

In [1]:
from FederatedLearning.learner import CNNModel

clients_num = 10 # 客户端总数
rotation_cycle = 5 # 角色轮换间隔轮数
max_round = 10 # 联邦学习轮数
model_class = CNNModel # 选用CNN模型

## 数据切分

In [2]:
import data_split
# 使用data_split模块中的函数获取PyTorch数据集
train_dataset = data_split.get_mnist_pytorch_dataset(train=True)
test_dataset = data_split.get_mnist_pytorch_dataset(train=False)
# test
# 使用正确的数据集对象创建数据加载器
client_dataloaders = data_split.create_client_dataloaders(train_dataset, clients_num // 2, 64) * 2
client_test_loader = data_split.create_client_dataloaders(test_dataset,clients_num // 2, 64) * 2

2025-10-19 10:08:39,106 - DGS_BCFL - INFO - 
切分5份数据集...
2025-10-19 10:08:39,108 - DGS_BCFL - INFO -  1 数据准备完成，样本数: 12000
2025-10-19 10:08:39,110 - DGS_BCFL - INFO -  2 数据准备完成，样本数: 12000
2025-10-19 10:08:39,111 - DGS_BCFL - INFO -  3 数据准备完成，样本数: 12000
2025-10-19 10:08:39,112 - DGS_BCFL - INFO -  4 数据准备完成，样本数: 12000
2025-10-19 10:08:39,113 - DGS_BCFL - INFO -  5 数据准备完成，样本数: 12000
2025-10-19 10:08:39,114 - DGS_BCFL - INFO - 
切分5份数据集...
2025-10-19 10:08:39,115 - DGS_BCFL - INFO -  1 数据准备完成，样本数: 2000
2025-10-19 10:08:39,115 - DGS_BCFL - INFO -  2 数据准备完成，样本数: 2000
2025-10-19 10:08:39,116 - DGS_BCFL - INFO -  3 数据准备完成，样本数: 2000
2025-10-19 10:08:39,116 - DGS_BCFL - INFO -  4 数据准备完成，样本数: 2000
2025-10-19 10:08:39,117 - DGS_BCFL - INFO -  5 数据准备完成，样本数: 2000


## 开始训练

In [7]:
from owner import Owner
from client import Client

# 初始化管理者，并获得初始化字典
owner = Owner(rotation_cycle=1, model_class=model_class)
main_dict = owner.get_main_dict()

# 初始化客户端以及身份
clients = [owner]
for i in range(clients_num):
    client_name = f"client_{i+1}"
    client = Client(epochs=2, client_name=client_name, data_loader=client_dataloaders[i], test_loader=client_test_loader[i], ModelClass=model_class, main_dict=main_dict)
    owner.join(client_name)
    clients.append(client)


import threading
for _ in range(max_round):
    t = [threading.Thread(target=client.run) for client in clients]
    _ = [i.start() for i in t]
    _ = [i.join() for i in t]


# 保存实验过程和结果
import pickle
with open("main_dict.pkl", "wb") as f:
    pickle.dump(main_dict, f)

2025-10-19 10:52:58,981 - DGS_BCFL - INFO - [client_5] 当前轮次角色: learner
2025-10-19 10:52:59,060 - DGS_BCFL - INFO - [client_6] 当前轮次角色: learner
2025-10-19 10:52:59,785 - DGS_BCFL - INFO - [client_7] 当前轮次角色: learner
2025-10-19 10:52:59,806 - DGS_BCFL - INFO - [client_8] 当前轮次角色: learner
2025-10-19 10:52:59,843 - DGS_BCFL - INFO - [client_9] 当前轮次角色: learner
2025-10-19 10:52:59,959 - DGS_BCFL - INFO - [client_10] 当前轮次角色: learner
2025-10-19 10:53:03,720 - DGS_BCFL - INFO - [client_6] 学习者开始本地训练...
2025-10-19 10:53:03,937 - DGS_BCFL - INFO - [client_8] 学习者开始本地训练...
2025-10-19 10:53:04,164 - DGS_BCFL - INFO - [client_5] 学习者开始本地训练...
2025-10-19 10:53:04,211 - DGS_BCFL - INFO - [client_9] 学习者开始本地训练...
2025-10-19 10:53:04,363 - DGS_BCFL - INFO - [client_10] 学习者开始本地训练...
2025-10-19 10:53:06,110 - DGS_BCFL - INFO - [client_7] 学习者开始本地训练...
2025-10-19 10:53:03,937 - DGS_BCFL - INFO - [client_8] 学习者开始本地训练...
2025-10-19 10:53:04,164 - DGS_BCFL - INFO - [client_5] 学习者开始本地训练...
2025-10-19 10:53:04,363 - DG

KeyboardInterrupt: 

    answer_challenge(c, authkey)
  File "/home/zyserver/miniconda3/envs/bcfl/lib/python3.12/multiprocessing/connection.py", line 962, in answer_challenge
    buf = self._recv(4)
          ^^^^^^^^^^^^^
  File "/home/zyserver/miniconda3/envs/bcfl/lib/python3.12/multiprocessing/connection.py", line 395, in _recv
    response = connection.recv_bytes(256)        # reject large message
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/zyserver/miniconda3/envs/bcfl/lib/python3.12/multiprocessing/connection.py", line 216, in recv_bytes
2025-10-19 10:53:47,274 - DGS_BCFL - INFO - [client_3] 当前全局模型 精度: 10.90% 损失: 2.2958
    chunk = read(handle, remaining)
            ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 104] Connection reset by peer
    buf = self._recv_bytes(maxlength)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/zyserver/miniconda3/envs/bcfl/lib/python3.12/multiprocessing/connection.py", line 430, in _recv_bytes
    buf = self._recv(4)
          ^^^^^^^^^^^^^
  Fi

## 计算结果可视化

In [ ]:
# 读取实验结果
main_dict = pickle.load(open("main_dict.pkl", "rb"))
# 进行可视化